# Combinations, search and exploration

https://excalidraw.com/#json=6016325231050752,W5D57AJn7nCdQ7MsmYvwJQ

In [1]:
from utils import render_excalidraw
from pprint import pprint, pformat

In [2]:
render_excalidraw(
    "https://excalidraw.com/#json=6016325231050752,W5D57AJn7nCdQ7MsmYvwJQ",
)

6 different actions


**INVENTORY**

* Glass
    * capactiy
    * current level
    * collection of glasses (state of the world)
* Goal to reach
* Pouring actions
    * emtpy
    * fill
    * transfer (X -> Y). Until Y is full or X is empty.
    
**Solution**

Sequence of steps to arrive to the desired state.

This is a **search** problem. In our "problem world" we will have an `explored` part and a `frontier`


**How to avoid infinite loops during search?**

* Shortest path first
* Don't re-explore

In [19]:
def pour_problem(X, Y, goal, start=(0, 0)):
    """
    X and Y are the capacity of glasses; (x, y) is current fill levels
    and represents a state. The goal is a level that can be in either glass.
    Start at start state and follow successors until we reach the goal.
    Keep track of frontier and previously explored; fail when no frontier.
    """
    # start with both glasses empty (0)
    if goal in start:
        return [start]
    
    # creat e a set to save the states we have already explored
    explored = set()
    
    # already explored world
    frontier = [[start]]
    
    # as long as we have a frontier to explore...
    while frontier:
        
        # get one of the paths
        path = frontier.pop(0)
        print(f"Doing path = {pformat(path)}\n")
        
        # get the glasses state at the end of that path
        (x, y) = path[-1]
        print(f"\nSearching for succesor steps\n")
        
        # based on the glasses state, find what possibilites
        # we have as "next step"
        for (state, action) in successors(x, y, X, Y).items():
            
            print(f"Action -> {action} will take us to State -> {state}\n")
            
            if state not in explored:
                print("State not explored")
                
                explored.add(state)
                print(f"Already explored:\n{pformat(explored)}\n")
                
                # update the path with the whole path we had before
                # + the new pair of action-state
                path2 = path + [action, state]
                print(f"Path2 = {path2}\n")
                
                # check if this new step has made us reach the goal
                if goal in state:
                    print(f"[!!] Found goal {goal} in state {state}\n")
                    return path2
                
                # 
                else:
                    frontier.append(path2)
                    print(f"Current frontier\n{pformat(frontier)}\n")
            else:
                print(f"State {state} already explored! Skipping...\n")
                
    return Fail


Fail = []

In [20]:
def successors(x, y, X, Y):
    """
    Return a dict of {state: action} pairs describing what can be reached from
    the (x, y) state, and how.
    """
    assert x <= X and y <= Y  # (x, y) is glass levels; X and Y are glass sizes
    return {
        ((0, y + x) if x + y <= Y else (x - (Y - y), y + (Y - y))): "X->Y",
        ((x + y, 0) if x + y <= X else (x + (X - x), y - (X - x))): "X<-Y",
        (X, y): "fill X",
        (x, Y): "fill Y",
        (0, y): "empty x",
        (x, 0): "empty Y",
    }


pour_problem(9, 4, 6)

Doing path = [(0, 0)]


Searching for succesor steps

Action -> empty Y will take us to State -> (0, 0)

State not explored
Already explored:
{(0, 0)}

Path2 = [(0, 0), 'empty Y', (0, 0)]

Current frontier
[[(0, 0), 'empty Y', (0, 0)]]

Action -> fill X will take us to State -> (9, 0)

State not explored
Already explored:
{(9, 0), (0, 0)}

Path2 = [(0, 0), 'fill X', (9, 0)]

Current frontier
[[(0, 0), 'empty Y', (0, 0)], [(0, 0), 'fill X', (9, 0)]]

Action -> fill Y will take us to State -> (0, 4)

State not explored
Already explored:
{(9, 0), (0, 0), (0, 4)}

Path2 = [(0, 0), 'fill Y', (0, 4)]

Current frontier
[[(0, 0), 'empty Y', (0, 0)],
 [(0, 0), 'fill X', (9, 0)],
 [(0, 0), 'fill Y', (0, 4)]]

Doing path = [(0, 0), 'empty Y', (0, 0)]


Searching for succesor steps

Action -> empty Y will take us to State -> (0, 0)

State (0, 0) already explored! Skipping...

Action -> fill X will take us to State -> (9, 0)

State (9, 0) already explored! Skipping...

Action -> fill Y will take us

[(0, 0),
 'fill X',
 (9, 0),
 'X->Y',
 (5, 4),
 'empty Y',
 (5, 0),
 'X->Y',
 (1, 4),
 'empty Y',
 (1, 0),
 'X->Y',
 (0, 1),
 'fill X',
 (9, 1),
 'X->Y',
 (6, 4)]

**Bridge problem**

We have 4 people with a torchlight on one side of the bridge. Each of them takes a different time to reach the other side.

They have to travel in pairs with the torchlight.


Representing state:

```
(here, there, t)
```

* `here`: people on one side. **frozenset**
* `there`: people on the other side. **frozenset**
* `t`: elapsed time.

In [ ]:
# exercise
# create the successors function

MY solution

In [ ]:
# User Instructions
#
# Write a function, bsuccessors(state), that takes a state as input
# and returns a dictionary of {state:action} pairs.
#
# A state is a (here, there, t) tuple, where here and there are
# frozensets of people (indicated by their times), and potentially
# the 'light,' t is a number indicating the elapsed time.
#
# An action is a tuple (person1, person2, arrow), where arrow is
# '->' for here to there or '<-' for there to here. When only one
# person crosses, person2 will be the same as person one, so the
# action (2, 2, '->') means that the person with a travel time of
# 2 crossed from here to there alone.


def bsuccessors(state):
    """
    Return a dict of {state:action} pairs. A state is a (here, there, t) tuple,
    where here and there are frozensets of people (indicated by their times) and/or
    the 'light', and t is a number indicating the elapsed time. Action is represented
    as a tuple (person1, person2, arrow), where arrow is '->' for here to there and
    '<-' for there to here.
    """

    here, there, t = state

    res = {}

    if "light" in here:

        for p1 in here:
            if p1 != "light":
                for p2 in here:
                    if p2 != "light":

                        res[
                            (
                                here - frozenset([p1, p2, "light"]),
                                there | frozenset([p1, p2, "light"]),
                                t + max(p1, p2),
                            )
                        ] = (p1, p2, "->")

    else:

        for p1 in there:

            if p1 != "light":
                for p2 in there:
                    if p2 != "light":

                        res[
                            (
                                here | frozenset([p1, p2, "light"]),
                                there - frozenset([p1, p2, "light"]),
                                t + max(p1, p2),
                            )
                        ] = (p1, p2, "<-")

    return res

Peter's solution

In [40]:
def bsuccessors(state):
    """
    Return a dict of {state:action} pairs. A state is a (here, there, t) tuple,
    where here and there are frozensets of people (indicated by their times) and/or
    the 'light', and t is a number indicating the elapsed time. Action is represented
    as a tuple (person1, person2, arrow), where arrow is '->' for here to there and
    '<-' for there to here.
    """

    here, there, t = state
    #print(state)
    
    if "light" in here:
        #              remove 2p here -> there          add 2p here -> there          # time = time + slowest    # action (person 'a' and 'b' move in '->' direction
        return { (here - frozenset([a,b,"light"]), there | frozenset([a,b,"light"]), t + max(a,b)):             (a, b, "->")
                        for a in here if a is not "light"
                        for b in here if b is not "light"}
                    
    
    else:
        # this is the same as before, but the 2 people move in the opposite direction
        #              add 2p there -> here          remove 2p there -> here          # time = time + slowest    # action (person 'a' and 'b' move in '<-' direction
        return { (here | frozenset([a,b,"light"]), there - frozenset([a,b,"light"]), t + max(a,b)):             (a, b, "<-")
                        for a in there if a is not "light"
                        for b in there if b is not "light"}
    #                           ^^^^ there was a bug in Peter's code
    # it should be 'there'
                    
        

In [41]:
def test():

    assert bsuccessors((frozenset([1, "light"]), frozenset([]), 3)) == {
        (frozenset([]), frozenset([1, "light"]), 4): (1, 1, "->")
    }

    assert bsuccessors((frozenset([]), frozenset([2, "light"]), 0)) == {
        (frozenset([2, "light"]), frozenset([]), 2): (2, 2, "<-")
    }

    return "tests pass"


test()

'tests pass'

In [42]:
# ----------------
# User Instructions
#
# Write two functions, path_states and path_actions. Each of these
# functions should take a path as input. Remember that a path is a
# list of [state, action, state, action, ... ]
#
# path_states should return a list of the states. in a path, and
# path_actions should return a list of the actions.


def path_states(path):
    "Return a list of states in this path."
    return path[0::2] # even numbers


def path_actions(path):
    "Return a list of actions in this path."
    return  path[1::2] # odd numbers


def test():
    testpath = [
        (frozenset([1, 10]), frozenset(["light", 2, 5]), 5),  # state 1
        (5, 2, "->"),  # action 1
        (frozenset([10, 5]), frozenset([1, 2, "light"]), 2),  # state 2
        (2, 1, "->"),  # action 2
        (frozenset([1, 2, 10]), frozenset(["light", 5]), 5),
        (5, 5, "->"),
        (frozenset([1, 2]), frozenset(["light", 10, 5]), 10),
        (5, 10, "->"),
        (frozenset([1, 10, 5]), frozenset(["light", 2]), 2),
        (2, 2, "->"),
        (frozenset([2, 5]), frozenset([1, 10, "light"]), 10),
        (10, 1, "->"),
        (frozenset([1, 2, 5]), frozenset(["light", 10]), 10),
        (10, 10, "->"),
        (frozenset([1, 5]), frozenset(["light", 2, 10]), 10),
        (10, 2, "->"),
        (frozenset([2, 10]), frozenset([1, 5, "light"]), 5),
        (5, 1, "->"),
        (frozenset([2, 10, 5]), frozenset([1, "light"]), 1),
        (1, 1, "->"),
    ]
    assert path_states(testpath) == [
        (frozenset([1, 10]), frozenset(["light", 2, 5]), 5),  # state 1
        (frozenset([10, 5]), frozenset([1, 2, "light"]), 2),  # state 2
        (frozenset([1, 2, 10]), frozenset(["light", 5]), 5),
        (frozenset([1, 2]), frozenset(["light", 10, 5]), 10),
        (frozenset([1, 10, 5]), frozenset(["light", 2]), 2),
        (frozenset([2, 5]), frozenset([1, 10, "light"]), 10),
        (frozenset([1, 2, 5]), frozenset(["light", 10]), 10),
        (frozenset([1, 5]), frozenset(["light", 2, 10]), 10),
        (frozenset([2, 10]), frozenset([1, 5, "light"]), 5),
        (frozenset([2, 10, 5]), frozenset([1, "light"]), 1),
    ]
    assert path_actions(testpath) == [
        (5, 2, "->"),  # action 1
        (2, 1, "->"),  # action 2
        (5, 5, "->"),
        (5, 10, "->"),
        (2, 2, "->"),
        (10, 1, "->"),
        (10, 10, "->"),
        (10, 2, "->"),
        (5, 1, "->"),
        (1, 1, "->"),
    ]
    return "tests pass"


test()

'tests pass'

In [43]:
def elapsed_time(path):
    return path[-1][2]


def bridge_problem(here):
    """Modify this to test for goal later: after pulling a state off frontier,
    not when we are about to put it on the frontier."""
    ## modify code below
    here = frozenset(here) | frozenset(["light"])
    explored = set()  # set of states we have visited
    # State will be a (people-here, people-there, time-elapsed)
    frontier = [[(here, frozenset(), 0)]]  # ordered list of paths we have blazed
    if not here:
        return frontier[0]
    while frontier:
        path = frontier.pop(0)
        for (state, action) in bsuccessors(path[-1]).items():
            if state not in explored:
                here, there, t = state
                explored.add(state)
                path2 = path + [action, state]
                if not here:  ## That is, nobody left here
                    return path2
                else:
                    frontier.append(path2)
                    frontier.sort(key=elapsed_time)
    return []


def test():
    assert (
        bridge_problem(frozenset((1, 2),))[
            -1
        ][-1]
        == 2
    )  # the [-1][-1] grabs the total elapsed time
    assert (
        bridge_problem(frozenset((1, 2, 5, 10),))[
            -1
        ][-1]
        == 19
    )
    return "tests pass"


test()

'tests pass'

In the proposed solution, the path is search based on **cost** (time elapsed) and each point, not taking into account the final cost.

One way to fix it is **test later**, when the goal is reached, put the path back in the frontier. If we try to exhaust the frontier, if it's not finite we will never solve the search.

In [50]:
# -----------------
# User Instructions
#
# Modify the bridge_problem(here) function so that it
# tests for goal later: after pulling a state off the
# frontier, not when we are about to put it on the
# frontier.


def elapsed_time(path):
    return path[-1][2]


def bridge_problem(here):
    """Modify this to test for goal later: after pulling a state off frontier,
    not when we are about to put it on the frontier."""
    ## modify code below
    here = frozenset(here) | frozenset(["light"])
    explored = set()  # set of states we have visited
    # State will be a (people-here, people-there, time-elapsed)
    frontier = [[(here, frozenset(), 0)]]  # ordered list of paths we have blazed
    if not here:
        return frontier[0]
    while frontier:
        path = frontier.pop(0)
        state1 = path[-1]
        here1, there1, t1 = state1
        if not here1 or here1 == set(["light"]):  ## That is, nobody left here
            return path
        for (state, action) in bsuccessors(path[-1]).items():
            if state not in explored:
                here, there, t = state
                explored.add(state)
                path2 = path + [action, state]

                frontier.append(path2)
                frontier.sort(key=elapsed_time)
    return []


def test():
    assert (
        bridge_problem(frozenset((1, 2),))[
            -1
        ][-1]
        == 2
    )  # the [-1][-1] grabs the total elapsed time
    assert (
        bridge_problem(frozenset((1, 2, 5, 10),))[
            -1
        ][-1]
        == 17
    )
    return "tests pass"


test()

'tests pass'

In [52]:
import doctest

class TestBridge: """
>>> elapsed_time(bridge_problem([1,2,5,10]))
17

## There are two equally good solutions
>>> S1 = [(2, 1, '->'), (1, 1, '<-'), (5, 10, '->'), (2, 2, '<-'), (2, 1, '->')]
>>> S2 = [(2, 1, '->'), (2, 2, '<-'), (5, 10, '->'), (1, 1, '<-'), (2, 1, '->')]
>>> path_actions(bridge_problem([1,2,5,10])) in (S1, S2)
True

## Try some other problems
>>> path_actions(bridge_problem([1,2,5,10,15,20]))
[(2, 1, '->'), (1, 1, '<-'), (10, 5, '->'), (2, 2, '<-'), (2, 1, '->'), (1, 1, '<-'), (15, 20, '->'), (2, 2, '<-'), (2, 1, '->')]

>>> path_actions(bridge_problem([1,2,4,8,16,32]))
[(2, 1, '->'), (1, 1, '<-'), (8, 4, '->'), (2, 2, '<-'), (2, 1, '->'), (1, 1, '<-'), (16, 32, '->'), (2, 2, '<-'), (2, 1, '->')]

>>> [elapsed_time(bridge_problem([1,2,4,8,16][:N])) for N in range(6)]
[0, 1, 2, 7, 15, 28]

>>> [elapsed_time(bridge_problem([1,1,2,3,5,8,13,21][:N])) for N in range(8)]
[0, 1, 1, 2, 6, 12, 19, 30]

"""

doctest.testmod()

TestResults(failed=0, attempted=8)

Our current approach is not very optimal, it allows "infinite" loops. The state representation is also not good because we are keeping the current elapsed time as part of the state.

In [7]:
# -----------------
# User Instructions
#
# write a function, bsuccessors2 that takes a state as input
# and returns a dictionary of {state:action} pairs.
#
# The new representation for a path should be a list of
# [state, (action, total time), state, ... , ], though this
# function will just return {state:action} pairs and will
# ignore total time.
#
# The previous bsuccessors function is included for your reference.


def bsuccessors2(state):
    """Return a dict of {state:action} pairs. A state is a
    (here, there) tuple, where here and there are frozensets
    of people (indicated by their travel times) and/or the light."""
    # your code here

    here, there = state
    if "light" in here:
        return dict(
            (
                (here - frozenset([a, b, "light"]), there | frozenset([a, b, "light"])),
                (a, b, "->"),
            )
            for a in here
            if a is not "light"
            for b in here
            if b is not "light"
        )
    else:
        return dict(
            (
                (here | frozenset([a, b, "light"]), there - frozenset([a, b, "light"])),
                (a, b, "<-"),
            )
            for a in there
            if a is not "light"
            for b in there
            if b is not "light"
        )


def bsuccessors(state):
    """Return a dict of {state:action} pairs.  A state is a (here, there, t) tuple,
    where here and there are frozensets of people (indicated by their times) and/or
    the light, and t is a number indicating the elapsed time."""
    here, there, t = state
    if "light" in here:
        return dict(
            (
                (
                    here - frozenset([a, b, "light"]),
                    there | frozenset([a, b, "light"]),
                    t + max(a, b),
                ),
                (a, b, "->"),
            )
            for a in here
            if a is not "light"
            for b in here
            if b is not "light"
        )
    else:
        return dict(
            (
                (
                    here | frozenset([a, b, "light"]),
                    there - frozenset([a, b, "light"]),
                    t + max(a, b),
                ),
                (a, b, "<-"),
            )
            for a in there
            if a is not "light"
            for b in there
            if b is not "light"
        )


def test():
    here1 = frozenset([1, "light"])
    there1 = frozenset([])

    here2 = frozenset([1, 2, "light"])
    there2 = frozenset([3])

    assert bsuccessors2((here1, there1)) == {
        (frozenset([]), frozenset([1, "light"])): (1, 1, "->")
    }
    assert bsuccessors2((here2, there2)) == {
        (frozenset([1]), frozenset(["light", 2, 3])): (2, 2, "->"),
        (frozenset([2]), frozenset([1, 3, "light"])): (1, 1, "->"),
        (frozenset([]), frozenset([1, 2, 3, "light"])): (2, 1, "->"),
    }
    return "tests pass"


test()

'tests pass'

Now we need some functions to calculate the cost of actions and paths

In [18]:
# -----------------
# User Instructions
# 
# Write a function, path_cost, which takes a path as input 
# and returns the total cost associated with that path. 
# Remember that paths will obey the convention
# path = (state, (action, total_cost), state, ...) 
#
# If a path is less than length 3, your function should
# return a cost of 0.

def path_cost(path):
    """The total cost of a path (which is stored in a tuple
    with the final action."""
    # path = (state, (action, total_cost), state, ... )
    if len(path) < 3:
        return 0
    else:
        action, total_cost = path[-2]
        return total_cost
        # acts = path[1::2]
        # ???
        
def bcost(action):
    """Returns the cost (a number) of an action in the
    bridge problem."""
    # An action is an (a, b, arrow) tuple; a and b are 
    # times; arrow is a string. 
    a, b, arrow = action
    return max(a,b)# ???

def test():
    assert path_cost(('fake_state1', ((2, 5, '->'), 5), 'fake_state2')) == 5
    assert path_cost(('fs1', ((2, 1, '->'), 2), 'fs2', ((3, 4, '<-'), 6), 'fs3')) == 6
    assert bcost((4, 2, '->'),) == 4
    assert bcost((3, 10, '<-'),) == 10
    return 'tests pass'

test()

'tests pass'

Putting it together

In [21]:
def bridge_problem2(here):
    here = frozenset(here) | frozenset(["light"])
    explored = set()  # set of states we have visited
    # State will be a (peoplelight_here, peoplelight_there) tuple
    # E.g. ({1, 2, 5, 10, "light"}, {})
    frontier = [[(here, frozenset())]]  # ordered list of paths we have blazed
    while frontier:
        path = frontier.pop(0)
        here1, there1 = state1 = final_state(path)
        if not here1 or (len(here1) == 1 and "light" in here1):
            return path
        explored.add(state1)
        pcost = path_cost(path)
        for (state, action) in bsuccessors2(state1).items():
            if state not in explored:
                total_cost = pcost + bcost(action)
                path2 = path + [(action, total_cost), state]
                add_to_frontier(frontier, path2)
    return Fail


Fail = []


def final_state(path):
    return path[-1]


def add_to_frontier(frontier, path):
    """Add path to frontier, replacing costlier path if there is one."""
    # (This could be done more efficiently.)
    # Find if there is an old path to the final state of this path.
    old = None
    for i, p in enumerate(frontier):
        if final_state(p) == final_state(path):
            old = i
            break
    if old is not None and path_cost(frontier[old]) < path_cost(path):
        return  # Old path was better; do nothing
    elif old is not None:
        del frontier[old]  # Old path was worse; delete it
    # Now add the new path and re-sort
    frontier.append(path)

In [22]:
bridge_problem2([1, 2, 4, 8, 16, 32])

bridge_problem2([1, 2, 5, 10, 15, 20])

[(frozenset({1, 10, 15, 2, 20, 5, 'light'}), frozenset()),
 ((2, 1, '->'), 2),
 (frozenset({5, 10, 15, 20}), frozenset({1, 2, 'light'})),
 ((2, 2, '<-'), 4),
 (frozenset({10, 15, 2, 20, 5, 'light'}), frozenset({1})),
 ((10, 5, '->'), 14),
 (frozenset({2, 15, 20}), frozenset({1, 10, 5, 'light'})),
 ((1, 1, '<-'), 15),
 (frozenset({1, 15, 2, 20, 'light'}), frozenset({5, 10})),
 ((2, 1, '->'), 17),
 (frozenset({15, 20}), frozenset({1, 10, 2, 5, 'light'})),
 ((2, 2, '<-'), 19),
 (frozenset({15, 2, 20, 'light'}), frozenset({1, 5, 10})),
 ((15, 20, '->'), 39),
 (frozenset({2}), frozenset({1, 10, 15, 20, 5, 'light'})),
 ((1, 1, '<-'), 40),
 (frozenset({1, 2, 'light'}), frozenset({5, 10, 15, 20})),
 ((2, 1, '->'), 42),
 (frozenset(), frozenset({1, 10, 15, 2, 20, 5, 'light'}))]